# 进展报告自动生成

基于项目文件（GitHubClient）调用大模型（LLM）自动生成项目进展报告。

### 调用 OpenAI GPT 大模型

相比 GitHub REST API ，OpenAI 提供的大模型相关 API 迭代速度快，且不够稳定。

**GPT-4 很难能够准确的生成 OpenAI Client 相关代码**。

因此，GitHubSentinel 项目中 LLM 相关调用代码由人类编写😁。


## Prompt 优化测试

基于 `GithubClient` 模块获取的 Repo 最新进展，先在 ChatGPT 中尝试获取可用的提示词（Prompt）方案。

-  **完整的ChatGPT 对话记录【GitHubSentinel 提示词优化】**：https://chatgpt.com/share/28524ea6-2bf3-4ebe-b7d9-9c1ba5f005d2
- 以下测试使用的 LangChain 项目文件为: `./daily_progress/langchain-ai_langchain/2024-08-18.md'`


### ChatGPT（GPT-4） 生成报告

**Langchain-AI/Langchain Daily Progress Report - 2024-08-18**

### 新增功能
1. **Langchain 模块添加**
   - 新增了Langchain Box套件及其文档加载器 (`langchain-box: add langchain box package and DocumentLoader` #25506)
   - 加入了新的社区提供者—Agentic RAG 示例 (`Community: Add Union provider - Agentic RAG example` #25509)
   - 引入了更多的异步测试标准 (`standard-tests[patch]: async variations of all tests` #25501)
   - 引入了对多种区块链的支持 (`community: add supported blockchains to Blockchain Document Loader` #25428)
   
2. **文档与API更新**
   - 更新了多个集成参考文档和Langchain版本的文档 (`docs: `integrations` reference update 9` #25511, `docs 0.3 release` #25459)
   - 增加了新的文档索引和数据加载方式的说明 (`[docs]: more indexing of document loaders` #25500)

### 主要改进
1. **测试与标准化**
   - 添加了更多嵌入标准测试 (`more embeddings standard tests` #25513)
   - 引入了JSON模式的标准测试 (`json mode standard test` #25497)
   - 新增了各种文档加载器的文档 (`[Doc] Add docs for `ZhipuAIEmbeddings`` #25467)

2. **框架和规则改进**
   - 对Langchain核心模块进行了Pydantic解析器修复 (`langchain-core: added pydantic parser fix for issue #24995` #25516)
   - 增加了B(bugbear) ruff规则以提高代码质量 (`core: Add B(bugbear) ruff rules` #25520)

3. **集成与兼容性**
   - 测试了Pydantic 2和Langchain 0.3的兼容性 (`openai[major] -- test with pydantic 2 and langchain 0.3` #25503)
   - 准备了向Pydantic 2迁移的根验证器升级 (`openai[patch]: Upgrade @root_validators in preparation for pydantic 2 migration` #25491)

### 修复问题
1. **错误修复**
   - 修正了文档中的错别字和错误消息 (`docs: Fix typo in openai llm integration notebook` #25492, `docs: fix Agent deprecation msg` #25464)
   - 解决了不同的搜索模式（向量与文本）产生不同结果的问题 (`Chroma search with vector and search with text get different result using the same embedding function` #25517)
   - 修复了使用AzureSearch vectorstore时的文档ID作为键的问题 (`community : [bugfix] Use document ids as keys in AzureSearch vectorstore` #25486)

2. **系统错误与异常处理**
   - 解决了在调用特定链时缺少输入键的错误 (`Raises ValueError: Missing some input keys: {'query'} everytime I invoke 'GraphCypherQAChain.from_llm' chain with query present as input keys` #25476)
   - 修正了未知类型 'ToolMessage' 的类型错误 (`TypeError: Got unknown type 'ToolMessage'.` #25490)

---

此简报详细总结了Langchain AI项目在2024年8月18日的最新进展，包括新增功能、主要改进和问题修复，确保团队成员了解最新的项目状态和即将到来的更新。


## 前置依赖 logger 模块

In [21]:
# src/logger.py
from loguru import logger
import sys

# Configure Loguru
logger.remove()  # Remove the default logger
logger.add(sys.stdout, level="DEBUG", format="{time} {level} {message}", colorize=True)
logger.add("logs/app.log", rotation="1 MB", level="DEBUG")

# Alias the logger for easier import
LOG = logger

# Make the logger available for import with the alias
__all__ = ["LOG"]

## LLM Class 

In [37]:
import os
from openai import OpenAI  # 导入OpenAI库用于访问GPT模型
# from logger import LOG  # 导入日志模块（演示时直接导入）

class LLM:
    def __init__(self, model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_API_BASE")):
        # 创建一个OpenAI客户端实例
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        # 确定使用的模型版本
        self.model = model
        # 配置日志文件，当文件大小达到1MB时自动轮转，日志级别为DEBUG
        LOG.add("daily_progress/llm_logs.log", rotation="1 MB", level="DEBUG")

    def generate_daily_report(self, markdown_content, dry_run=False):
        # 构建一个用于生成报告的提示文本，要求生成的报告包含新增功能、主要改进和问题修复
        prompt = f"以下是项目的最新进展，请根据功能合并同类项，形成一份简报，:\n\n{markdown_content}"
        system = "你是一名专业的GitHub项目管理助手，专门帮助用户对GitHub上开源项目的Issue,commit或Pull Request根据功能进行分类和摘要形成一份简报。至少包含：1）新增功能；2）主要改进；3）修复问题。请使用中文"
        
        if dry_run:
            # 如果启用了dry_run模式，将不会调用模型，而是将提示信息保存到文件中
            LOG.info("Dry run mode enabled. Saving prompt to file.")
            with open("daily_progress/prompt.txt", "w+") as f:
                f.write(prompt)
            LOG.debug("Prompt saved to daily_progress/prompt.txt")
            return "DRY RUN"

        # 日志记录开始生成报告
        LOG.info(f"Starting report generation using model: {self.model}.")
        
        try:
            # 调用OpenAI GPT模型生成报告
            response = self.client.chat.completions.create(
                model=self.model,  # 指定使用的模型版本
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt}  # 提交用户角色的消息
                ]
            )
            LOG.debug(f"{self.model} response: {response}")
            # 返回模型生成的内容
            return response.choices[0].message.content
        except Exception as e:
            # 如果在请求过程中出现异常，记录错误并抛出
            LOG.error("An error occurred while generating the report: {}", e)
            raise


## ReportGenerator Class

In [42]:
# src/report_generator.py

import os
from datetime import date, timedelta
# from logger import LOG  # 导入日志模块，用于记录日志信息

class ReportGenerator:
    def __init__(self, llm):
        self.llm = llm  # 初始化时接受一个LLM实例，用于后续生成报告

    def export_daily_progress(self, repo, updates):
        # 构建仓库的日志文件目录
        repo_dir = os.path.join('daily_progress', repo.replace("/", "_"))
        os.makedirs(repo_dir, exist_ok=True)  # 如果目录不存在则创建
        
        # 创建并写入日常进展的Markdown文件
        file_path = os.path.join(repo_dir, f'{date.today()}.md')
        with open(file_path, 'w') as file:
            file.write(f"# Daily Progress for {repo} ({date.today()})\n\n")
            file.write("\n## Issues\n")
            for issue in updates['issues']:
                file.write(f"- {issue['title']} #{issue['number']}\n")
            file.write("\n## Pull Requests\n")
            for pr in updates['pull_requests']:
                file.write(f"- {pr['title']} #{pr['number']}\n")
        return file_path

    def export_progress_by_date_range(self, repo, updates, days):
        # 构建目录并写入特定日期范围的进展Markdown文件
        repo_dir = os.path.join('daily_progress', repo.replace("/", "_"))
        os.makedirs(repo_dir, exist_ok=True)

        today = date.today()
        since = today - timedelta(days=days)  # 计算起始日期
        
        date_str = f"{since}_to_{today}"  # 格式化日期范围字符串
        file_path = os.path.join(repo_dir, f'{date_str}.md')
        
        with open(file_path, 'w') as file:
            file.write(f"# Progress for {repo} ({since} to {today})\n\n")
            file.write("\n## Issues Closed in the Last {days} Days\n")
            for issue in updates['issues']:
                file.write(f"- {issue['title']} #{issue['number']}\n")
            file.write("\n## Pull Requests Merged in the Last {days} Days\n")
            for pr in updates['pull_requests']:
                file.write(f"- {pr['title']} #{pr['number']}\n")
        
        LOG.info(f"Exported time-range progress to {file_path}")  # 记录导出日志
        return file_path

    def generate_daily_report(self, markdown_file_path):
        # 读取Markdown文件并使用LLM生成日报
        with open(markdown_file_path, 'r') as file:
            markdown_content = file.read()

        report = self.llm.generate_daily_report(markdown_content)  # 调用LLM生成报告

        report_file_path = os.path.splitext(markdown_file_path)[0] + "_report.md"
        with open(report_file_path, 'w+', encoding='utf-8') as report_file:
            report_file.write(report)  # 写入生成的报告

        LOG.info(f"Generated report saved to {report_file_path}")  # 记录生成报告日志

    def generate_report_by_date_range(self, markdown_file_path, days):
        # 生成特定日期范围的报告，流程与日报生成类似
        with open(markdown_file_path, 'r') as file:
            markdown_content = file.read()

        report = self.llm.generate_daily_report(markdown_content)

        report_file_path = os.path.splitext(markdown_file_path)[0] + f"_report.md"
        with open(report_file_path, 'w+', encoding='utf-8') as report_file:
            report_file.write(report)

        LOG.info(f"Generated report saved to {report_file_path}")  # 记录生成报告日志


## 调用 GPT-3.5-Turbo API 生成报告

In [43]:
# 实例化 LLM，并使用默认的 GPT-3.5-Turbo 模型
llm = LLM()

In [44]:
# 实例化 ReportGenerator
report_generator = ReportGenerator(llm)

In [45]:
# 生成 LangChain 项目最近一日报告
report_generator.generate_daily_report(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-08-20.md")

2024-08-21T00:05:59.790736+0800 INFO Starting report generation using model: gpt-3.5-turbo.
2024-08-21T00:06:17.025767+0800 DEBUG gpt-3.5-turbo response: ChatCompletion(id='chatcmpl-9yLf07sKKNc7NGZN9CKr2BFX2f2JK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='## 新增功能:\n1. 为丢失的GraphVectorStore扩展添加beta修饰符 #25562\n2. 向标准测试中指定`tool_choice` #25548\n3. 为`SingleFileFacebookMessengerChatLoader`的API文档添加改进 #25536\n4. 添加嵌入集成测试 #25508\n5. 为langchain-box添加langchain盒子包和DocumentLoader #25465\n6. 更新了`DocumentLoader` #25458\n\n## 主要改进:\n1. 更新了模型用于llama测试 #25542\n2. 改进了文档中的QianfanLLMEndpoint和Tongyi输入文本 #25529\n3. 更新了基本URL #25524\n4. 修复了在create_stuff_documents_chain中使用`document_variable_name`无法覆盖上下文的问题 #25531\n\n## 修复问题:\n1. 修复UnionType类型变量替换问题 #25544\n2. 修复Databricks Vector Search演示笔记本问题 #25504\n3. 修复了在Azure中的json模式问题 #25488', role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1724169970, model='gpt-3.5-turbo-0125', obje

In [54]:
report_generator.generate_report_by_date_range(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-08-17_to_2024-08-20.md", days=3)

2024-08-21T00:15:55.517067+0800 INFO Starting report generation using model: gpt-3.5-turbo.
2024-08-21T00:16:09.912971+0800 DEBUG gpt-3.5-turbo response: ChatCompletion(id='chatcmpl-9yLoWZVXTov55p1JSBruQq3pLqjzp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='## 简报：langchain-ai/langchain 项目进展 (2024-08-17 至 2024-08-20)\n\n### 新增功能：\n- 添加 beta 装饰器到遗漏的 GraphVectorStore 扩展 #25562\n- 在标准测试中指定 tool_choice #25548\n- 为 `SearxSearchResults` 工具添加 args_schema #25350\n- 添加 embeddings 集成测试 #25508\n- 升级 @root_validator 的使用以与 pydantic 2 一致化 #25457\n\n### 主要改进：\n- 改进 `SingleFileFacebookMessengerChatLoader` 的 API 文档 #25536\n- 更新 llama 测试所使用的模型 #25542\n- 更新基础网址 #25524\n- 修复了无法使用 document_variable_name 覆盖 create_stuff_documents_chain 中上下文的问题 #25531\n- 修复 ChatOpenAI 在本地 LM Studio 服务器上无法运行的问题 #25545\n\n### 修复问题：\n- 修复 UnionType 类型变量替换的问题 #25544\n- 修复 "DocumentLoader" 对象缺少 nltk 包的问题 #25473\n- 修复 ChatZhipuAI 的 ToolMessage 的问题 #25547\n- 修复了 ChatOpenAI.wit

## 调用 GPT-4-Turbo API 生成报告

In [46]:
# 实例化 LLM，并使用指定的 GPT-4-Turbo 模型
gpt_4 = LLM(model="gpt-4o")

In [47]:
# 实例化 ReportGenerator, 并使用指定的 GPT-4-Turbo 模型
rg_gpt_4 = ReportGenerator(gpt_4)

In [48]:
# 生成 LangChain 项目最近一日报告
rg_gpt_4.generate_daily_report(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-08-20.md")

2024-08-21T00:07:29.290823+0800 INFO Starting report generation using model: gpt-4o.
2024-08-21T00:07:44.501548+0800 DEBUG gpt-4o response: ChatCompletion(id='chatcmpl-9yLgL4fXdUzXhylFYBU42fBVusJD6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**langchain-ai/langchain 项目进展简报（2024-08-20）**\n\n### 新增功能\n1. **新功能集成**\n   - 在核心模块和社区模块中，新增了对GraphVectorStore扩展具有Beta装饰器的功能。\n   - 添加了`ToolMessage`功能到ChatZhipuAI。\n   - 在`langchain-box`包中新增了`DocumentLoader`。\n\n### 主要改进\n1. **基础设施改进**\n   - 移除了CI树中的huggingface依赖。\n   - 更新了OpenAI和核心依赖版本分别到0.1.22和0.2.33。\n   - 升级了@root_validator的使用，以支持pydantic 2，涉及模块包括core, anthropic, voyageai, ai21, pinecone。\n\n2. **文档和示例更新**\n   - 增强了`SingleFileFacebookMessengerChatLoader`的API文档。\n   - 更新了`QianfanLLMEndpoint`和Tongyi的输入文本文档。\n   - 在文档中增加了安装nltk和beautifulsoup4的提示。\n   - 修正了Databricks向量搜索示例笔记本的文档。\n   - 在OpenAI集成笔记本中使用`.invoke`代替`__call__`。\n   - 修正了代理弃用消息的文档。\n   - 更新了文档中对`arxiv`页面的内容。\n   - 重新格式化了oai嵌入文档字符

In [55]:
rg_gpt_4.generate_report_by_date_range(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-08-17_to_2024-08-20.md", days=3)

2024-08-21T00:17:16.763639+0800 INFO Starting report generation using model: gpt-4o.
2024-08-21T00:17:36.407135+0800 DEBUG gpt-4o response: ChatCompletion(id='chatcmpl-9yLpobbSQ6su93Cp4ONifidg4YHyG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='# Langchain 项目进展简报（2024-08-17 至 2024-08-20）\n\n## 新增功能\n1. **工具与集成**\n   - 添加了 `ToolMessage` 用于 ChatZhipuAI (#25547)。\n   - 开发了 langchain-box 包，并新增了 `DocumentLoader` (#25465)。\n\n2. **模型与测试**\n   - 为 llama 测试更新了模型 (#25542)。\n   - 指定了标准测试中的 tool_choice (#25548)。\n\n## 主要改进\n1. **文档与示例**\n   - 改进了 `SingleFileFacebookMessengerChatLoader` 的API文档 (#25536)。\n   - 更新归纳指南 (#25408)。\n   - 修复了 Databricks 向量搜索演示笔记本 (#25504)。\n   - 使用 `.invoke` 而不是 `__call__` 在 OpenAI 集成笔记本中 (#25494)。\n   - 修复 Agent 废弃消息 (#25464)。\n   - 修复 mimetype 解析器 docstring (#25463)。\n   - 更新了 `arxiv` 页面 (#25450)。\n\n2. **依赖与工具**\n   - 删除 CI 树中的 huggingface (#25559)。\n   - 为 QianfanLLMEndpoint 和 Tongyi 修复了输入文本 (#25529)。\n   - 在 `o

In [49]:
moon = LLM(model="moonshot-v1-8k",api_key=os.getenv("MOONSHOT_API_KEY"), base_url=os.getenv("MOONSHOT_API_BASE"))

In [50]:
rg_moon = ReportGenerator(moon)

In [51]:
rg_moon.generate_daily_report(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-08-20.md")

2024-08-21T00:10:04.893459+0800 INFO Starting report generation using model: moonshot-v1-8k.
2024-08-21T00:10:19.038200+0800 DEBUG moonshot-v1-8k response: ChatCompletion(id='chatcmpl-66c4bfdcd669133bf8443621', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='### 项目简报：langchain-ai/langchain (2024-08-20)\n\n#### 新增功能\n- 引入了beta装饰器，用于扩展GraphVectorStore（#25562）。\n- 添加了ToolMessage，用于ChatZhipuAI（#25547）。\n- 增加了embeddings集成测试（#25508）。\n- 新增langchain box包和DocumentLoader（#25465, #25458）。\n\n#### 主要改进\n- 改进了`SingleFileFacebookMessengerChatLoader`的API文档（#25536）。\n- 更新了base URL（#25524）。\n- 增加了安装nltk的提示（#25519）。\n- 增加了安装beautifulsoup4的提示（#25518）。\n- 更新了Databricks Vector Search演示笔记本（#25504）。\n- 更新了openai核心依赖（#25502, #25496）。\n- 使用.invoke代替__call__在openai集成笔记本中（#25494）。\n- 修复了Azure的json模式问题（#25488）。\n- 更新了文档，包括Agent弃用消息和mimetype解析器文档字符串（#25464, #25463）。\n- 升级了@root_validator的使用，以与pydantic 2保持一致（#25457, #25455, #25454, #25453）。\n\n#### 修复问题\n- 从CI树中

In [56]:
rg_moon.generate_report_by_date_range(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-08-17_to_2024-08-20.md", days=3)

2024-08-21T00:19:11.585569+0800 INFO Starting report generation using model: moonshot-v1-8k.
2024-08-21T00:19:32.375073+0800 DEBUG moonshot-v1-8k response: ChatCompletion(id='chatcmpl-66c4c1ff48607124c76e9acb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='## 项目简报：langchain-ai/langchain (2024-08-17 至 2024-08-20)\n\n### 新增功能\n- 为GraphVectorStore扩展添加了beta装饰器，以支持更多功能（#25562）。\n- 引入了ToolMessage，用于ChatZhipuAI（#25547）。\n- 添加了对`SingleFileFacebookMessengerChatLoader`的API文档改进（#25536）。\n- 新增了embeddings集成测试（#25508）。\n\n### 主要改进\n- 从CI树中移除了huggingface（#25559）。\n- 测试了聊天提示的序列化和反序列化（#25557）。\n- 在标准测试中指定了tool_choice（#25548）。\n- 更新了用于llama测试的模型（#25542）。\n- 更新了基础URL（#25524）。\n- 添加了安装nltk的提示（#25519）。\n- 添加了安装beautifulsoup4的提示（#25518）。\n- 更新了core依赖（#25502）。\n- 发布了0.2.33版本（#25498）。\n- 发布了0.1.22版本（#25496）。\n- 改进了openai集成笔记本中的调用方式（#25494）。\n- 修复了Azure的json模式问题（#25488）。\n- 添加了langchain box包和DocumentLoader（#25465, #25458）。\n- 升级了多个库以与pydantic 2保持一致性（#25457

### Homework: 与ChatGPT深度对话，尝试使用 System role 提升报告质量和稳定性